In [ ]:
!pip install -r requirements.txt

In [ ]:
!python /opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --list models.lst

In [ ]:
from __future__ import print_function
import os
import sys
import time
import logging as log
from argparse import ArgumentParser, SUPPRESS

import cv2
import numpy as np

from openvino.inference_engine import IECore
from utils.codec import CTCCodec

import sys
from PIL import Image
import PIL
import io
import IPython.display
from IPython.display import clear_output



def build_argparser():
    parser = ArgumentParser(add_help=False)
    args = parser.add_argument_group('Options')
    args.add_argument('-h', '--help', action='help', default=SUPPRESS,
                      help='Show this help message and exit.')
    args.add_argument("-m", "--model", type=str, required=True,
                      help="Path to an .xml file with a trained model.")
    args.add_argument("-i", "--input", type=str, required=True,
                      help="Required. Path to an image to infer")
    args.add_argument("-d", "--device", type=str, default="CPU",
                      help="Optional. Specify the target device to infer on; CPU, GPU, FPGA, HDDL, MYRIAD or HETERO: is "
                           "acceptable. The sample will look for a suitable plugin for device specified. Default "
                           "value is CPU")
    args.add_argument("-ni", "--number_iter", type=int, default=1,
                      help="Optional. Number of inference iterations")
    args.add_argument("-cl", "--charlist", type=str, default=os.path.join(os.path.dirname(__file__), "data/kondate_nakayosi_char_list.txt"), help="Path to the decoding char list file")
    return parser


def get_characters():
    '''Get characters'''
    charlist = "data/kondate_nakayosi_char_list.txt"
    with open(charlist, 'r', encoding='utf-8') as f:
        return ''.join(line.strip('\n') for line in f)


def preprocess_input(image_name, height, width):
    src = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    ratio = float(src.shape[1]) / float(src.shape[0])
    tw = int(height * ratio)
    rsz = cv2.resize(src, (tw, height), interpolation=cv2.INTER_AREA).astype(np.float32)
    # [h,w] -> [c,h,w]
    img = rsz[None, :, :]
    _, h, w = img.shape
    # right edge padding
    pad_img = np.pad(img, ((0, 0), (0, height - h), (0, width -  w)), mode='edge')
    return pad_img


def main(image_path):
    log.basicConfig(format="[ %(levelname)s ] %(message)s", level=log.INFO, stream=sys.stdout)
#    args = build_argparser().parse_args()

    # Plugin initialization
    ie = IECore()
    # Read IR
    log.info("Loading network")
    model = "intel/handwritten-japanese-recognition-0001/FP32/handwritten-japanese-recognition-0001.xml"
    net = ie.read_network(model, os.path.splitext(model)[0] + ".bin")

    assert len(net.inputs) == 1, "Demo supports only single input topologies"
    assert len(net.outputs) == 1, "Demo supports only single output topologies"

    log.info("Preparing input/output blobs")
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))

    characters = get_characters()
    codec = CTCCodec(characters)
    assert len(codec.characters) == net.outputs[out_blob].shape[2], "The text recognition model does not correspond to decoding character list"

    input_batch_size, input_channel, input_height, input_width= net.inputs[input_blob].shape

    # Read and pre-process input image (NOTE: one image only)
    input_path = image_path
    input_image = preprocess_input(input_path, height=input_height, width=input_width)[None,:,:,:]
    assert input_batch_size == input_image.shape[0], "The net's input batch size should equal the input image's batch size "
    assert input_channel == input_image.shape[1], "The net's input channel should equal the input image's channel"

    # Loading model to the plugin
    log.info("Loading model to the plugin")
    exec_net = ie.load_network(network=net, device_name="CPU")

    # Start sync inference
    number_iter = 1
    log.info("Starting inference ({} iterations)".format(number_iter))
    infer_time = []
    for i in range(number_iter):
        img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
        f = io.BytesIO()
        PIL.Image.fromarray(img).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))
        
        t0 = time.time()
        preds = exec_net.infer(inputs={input_blob: input_image})
        preds = preds[out_blob]
        result = codec.decode(preds)
        print(result)
        infer_time.append((time.time() - t0) * 1000)

    log.info("Average throughput: {} ms".format(np.average(np.asarray(infer_time))))

    sys.exit()

In [ ]:
main("data/test.png")

In [ ]:
main("myname.png")

In [ ]:
main("intelkk.png")

In [ ]:
main("techc.png")

In [ ]:
main("sentence.png")